In [1]:
import numpy as np
from numpy import f2py
import time
import scipy
from scipy.io import loadmat

## Forward Model
2D gravity anomaly calculation for sediment density inversion. First we create the forward model than detail the inverse method used to describe the density and depth of the different interfaces.

In [5]:
!f2py -c /Users/andrew/projects/gravity-inversion/gravity/HD_gravity_forward_calc/talwani2_comp.F -m talwani
!f2py -c /Users/andrew/projects/gravity-inversion/gravity/HD_gravity_forward_calc/plouff3_comp.F -m plouff3

import talwani
import plouff3

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "talwani" sources
f2py options: []
f2py:> /var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmplc0zq1qd/src.macosx-10.15-x86_64-3.9/talwanimodule.c
creating /var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmplc0zq1qd/src.macosx-10.15-x86_64-3.9
Reading fortran codes...
	Reading file '/Users/andrew/projects/gravity-inversion/gravity/HD_gravity_forward_calc/talwani2_comp.F' (format:fix,strict)
Post-processing...
	Block: talwani
			Block: talwani2_comp
Post-processing (stage 2)...
Building modules...
	Building module "talwani"...
		Constructing wrapper function "talwani2_comp"...
		  talwani2_comp(g,xobs,xp,zp,drho,nv,[mx,nx])
	Wrote C/API module "talwani" to file "/var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmplc0zq1qd/sr

/usr/local/bin/gfortran -Wall -g -Wall -g -undefined dynamic_lookup -bundle /var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmplc0zq1qd/var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmplc0zq1qd/src.macosx-10.15-x86_64-3.9/talwanimodule.o /var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmplc0zq1qd/var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmplc0zq1qd/src.macosx-10.15-x86_64-3.9/fortranobject.o /var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmplc0zq1qd/Users/andrew/projects/gravity-inversion/gravity/HD_gravity_forward_calc/talwani2_comp.o -L/usr/local/Cellar/gcc/10.2.0_4/lib/gcc/10/gcc/x86_64-apple-darwin19/10.2.0 -L/usr/local/Cellar/gcc/10.2.0_4/lib/gcc/10/gcc/x86_64-apple-darwin19/10.2.0/../../.. -L/usr/local/Cellar/gcc/10.2.0_4/lib/gcc/10/gcc/x86_64-apple-darwin19/10.2.0/../../.. -L/usr/local/lib -L/usr/local/opt/openssl@1.1/lib -L/usr/local/opt/sqlite/lib -L/usr/local/opt/tcl-tk/lib -lgfortran -o ./talwani.cpython-39-darwin.so
Removing build directory /var/folders/w2/1bng

/usr/local/bin/gfortran -Wall -g -Wall -g -undefined dynamic_lookup -bundle /var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmpi9l1my_l/var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmpi9l1my_l/src.macosx-10.15-x86_64-3.9/plouff3module.o /var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmpi9l1my_l/var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmpi9l1my_l/src.macosx-10.15-x86_64-3.9/fortranobject.o /var/folders/w2/1bng91rn3fzg3vv34rh2z8qr0000gn/T/tmpi9l1my_l/Users/andrew/projects/gravity-inversion/gravity/HD_gravity_forward_calc/plouff3_comp.o -L/usr/local/Cellar/gcc/10.2.0_4/lib/gcc/10/gcc/x86_64-apple-darwin19/10.2.0 -L/usr/local/Cellar/gcc/10.2.0_4/lib/gcc/10/gcc/x86_64-apple-darwin19/10.2.0/../../.. -L/usr/local/Cellar/gcc/10.2.0_4/lib/gcc/10/gcc/x86_64-apple-darwin19/10.2.0/../../.. -L/usr/local/lib -L/usr/local/opt/openssl@1.1/lib -L/usr/local/opt/sqlite/lib -L/usr/local/opt/tcl-tk/lib -lgfortran -o ./plouff3.cpython-39-darwin.so
Removing build directory /var/folders/w2/1bng9

In [24]:
polargap_HD = loadmat('/Users/andrew/projects/gravity-inversion/gravity/HD_gravity_forward_calc/polargap_HD.mat')
A = loadmat('/Users/andrew/projects/gravity-inversion/gravity/HD_gravity_forward_calc/BM_5km.mat')
boug_goce5000=loadmat('/Users/andrew/projects/gravity-inversion/gravity/HD_gravity_forward_calc/boug_goce5000.mat')

_

array([[-8.50000000e+05, -4.50000000e+05,  1.35317539e+03,
        -8.86482056e+02],
       [-8.45000000e+05, -4.50000000e+05,  1.33329418e+03,
        -7.08194702e+02],
       [-8.40000000e+05, -4.50000000e+05,  1.31304470e+03,
        -5.96590576e+02],
       ...,
       [ 1.40000000e+05,  1.50000000e+05,  2.85405717e+03,
         4.03899902e+02],
       [ 1.45000000e+05,  1.50000000e+05,  2.87182771e+03,
         4.92544189e+02],
       [ 1.50000000e+05,  1.50000000e+05,  2.89440342e+03,
         5.25711914e+02]])

In [41]:
S = boug_goce5000['boug_goce5000'].shape;
XI = np.reshape(boug_goce5000['boug_goce5000'][:,:,0],(S[0]*S[1],1))
YI = np.reshape(boug_goce5000['boug_goce5000'][:,:,1],(S[0]*S[1],1))
ZI = np.reshape(boug_goce5000['boug_goce5000'][:,:,2],(S[0]*S[1],1))
F = scatteredInterpolant(XI,YI,ZI,'natural');


In [40]:
boug_goce5000['boug_goce5000'].transpose().shape

(3, 201, 121)

In [215]:
def msplit(m,dlevel=0):
    r"""
    this function is used to split an earth model
    """
    
    m1 = np.ones((m.shape[0],m.shape[1]))*1e-10
    m2 = m
    for i in range(m.shape[1]):
        m1[:,i] = m1[:,i]*(m.shape[1]+1-i)
        id1 = m[:,i]<0
        m1[id1,i] = m[id1,i]
        m2[id1,i] = i*1e-10
    m1 = m1+dlevel
    m2 = m2+dlevel
    return m1, m2

def twodpolyA(xobs,M,rho0,rhoback):
    r"""
    TWODPOLYA computes gravity anomaly Dg of polygons using Talwani line 
    integral. For details, see Talwani et al. (1959), JGR, 64(1), 49-59
    """
    N = len(rho0)
    mx = len(xobs)
    drho = lrho0-rhoback
    dg = np.empty((mx,N))
    dg[:]=np.nan
    for i in range(N):
        ind = np.argwhere(M[1:-2,i+1]==M[0:-3,i+1] &
                   M[1:-2,i+1]==M[2:-1,i+1])
        xp=M[:,0]
        xp[ind+1]=[]
        zp = M[:,i+1]
        zp[ind+1] = []
        zp[zp==0]=1e-10
        dg[:,i] = talwani2(xobs,xp,zp,drho[i])
    return sum(dg,1)*1e5
        
def mmodtwodpoly(m0):
    a = np.vstack((np.append(m0[0,0]-5000.,m0[0,1:]),m0,np.append(m0[-1,0]+5000,m0[-1,1:])))
    m = np.append((np.append(a[:,0],np.flip(a[:,0]))),a[0,0])
    S = m0.shape;
    for i in range(1,S[1]):
        b = np.append(np.append(a[:,i],np.flip(a[:,i])),a[1,i]);
        m = np.vstack((m,b));
    
    return m

def makep(nx,ny,dx,dy):
    x = np.arange(0,dx*(nx),dx)
    P = np.tile(x,ny)
    count = 0
    for i in range(len(P)):
        P[i:i+nx] = np.ones(nx)*count*dy
        count =count+1
        print(i)
    return P

In [ ]:
def gravcalc_HD(LN,dm,ρ_B,ρ_S,ρ_I=917):
    return

In [ ]:
def gravcalc_HD2(LN,dm,ρ_B,ρ_S,ρ_I=917):
    return

In [ ]:
def gravcalc_HD_3D(LN,dm,ρ_B,ρ_S,dm,ρ_I=917,dx=5000,dy=5000,flevel=4000):
    """
    Gravity ellipsoid referenced to the WGS1984 ellipsoid.
    """
    
    xx=np.arange(-850000,150000,dx)
    yy=np.arange(-450000,150000,dy)
    mx=np.shape(xx)
    my=np.shape(yy)
    rhoi = 918
    rhoc = 2670
    rho1 = [rhoi;rhoc]                 
    rho2 = rhoi-rhoc

    dn=150000
    Pp = makep(mx,my,dx,dy)
    Pp = [Pp(:,1) Pp(:,1)+dx Pp(:,2) Pp(:,2)+dy]
    Pobs = [data(:,8)-xx(1)+dx/2 data(:,9)-yy(1)+dy/2]
    [m01,m02] = msplit(-A['BM_5km'][3:-1],flevel);
    m01 = [m01 ones(my*mx,1)*flevel+1e-3];


    time_start = time.clock()
    g = plouff3(Pobs,Pp,m01,m02,rho1,rho2,dx,dy,dn)
    time_elapsed = (time.clock() - time_start)
    hmax = 20000
    hmin = 10000
    glpf = scipy.signal.fftfilt_1d(data(:,10),g,hmax,hmin).transpose()
    glpf = glpf*1e5;
    
    load boug_goce5000

S = size(boug_goce5000);
XI = reshape(boug_goce5000(:,:,1)',S(1)*S(2),1);
YI = reshape(boug_goce5000(:,:,2)',S(1)*S(2),1);
ZI = reshape(boug_goce5000(:,:,3)',S(1)*S(2),1);

F = scatteredInterpolant(XI,YI,ZI,'natural');
boug_goce = F(data(:,8),data(:,9));
    
    
    return

In [ ]:
mm = mmodtwodpoly([data(:,10) -data(:,13) -data(:,11)]);
id = isnan(mm(:,2));
mm(id,:) = [];

col = ['c','c','c','c','c'];
figure
subplot(211);
title(LN)

npolygon = size(mm,2);
for ip=1:npolygon-1
    zp = mm(:,ip+1);          % z-position
    fill(mm(:,1)/1000,-1*zp,col(ip)); 
    hold on
end
title(LN)

xlim([min(data(:,10))/1000 max(data(:,10))/1000])
ylabel('Elevation (m)')
grid on

id = 1:3:length(data);

% Gravity anomaly
subplot(212)
plot(data(id,10)/1000,data(id,5),'ko');
hold all
plot(data(:,10)/1000,glpf,'-','LineWidth',2,'Color',[0 0.447 0.741]);
plot(data(:,10)/1000,boug_goce,'-','LineWidth',2,'Color',[0.466 0.674 0.188]);
plot(data(:,10)/1000,gmodel,'-','LineWidth',2,'Color',[0.850 0.325 0.098]);
title(LN)

xlim([min(data(:,10))/1000 max(data(:,10))/1000])
xlabel('Distance along the profile (km)')
ylabel('Gravity anomaly (miligals)')
grid on
legend('Observed gravity','Gravity effect of ice and topography',...
       'Gravity effect of the Moho','Sum of modelled gravity')